In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

## Use decision tree as a rough way for splitting based on that feature
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn import tree

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

from constants import regions

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
len( regions )

13

In [3]:
sys.argv[1] = 1
sys.argv[2] = 1

In [6]:
d_num = int( sys.argv[1] )
data_type = ['N2', 'N3','REM', 'Wake', 'N1'][d_num]

expt_num = int( sys.argv[2] )

#2. Load the data based on data type ___________________________________________________________
df = joblib.load(os.path.join(os.path.abspath('..'), data_type + '_psd_normalised_data.pkl') )

single_region_df = df[[col for col in df.columns if col.endswith('_Central')]]
frequency_vals  = np.arange(0.5,40.125, 0.125)
single_region_df.columns = frequency_vals

folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'
paths = joblib.load(folder + data_type + '_paths.pkl')

groups = pd.Series(paths['s_p_id'])
s_class_list = pd.Series(paths['s_class_list'])
y = s_class_list.map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3})

#3 Generate the frequency band search space ____________________________________________________
width_slide_list = [(0.5,0.5)] 
for window_width in range(1, 15 ):
    width_slide_list.append((window_width, 0.5) )

#4 Calculate all of the features ________________________________________________________________
feature_dfs = []

for window_width_hz, window_slide_hz in super_reduced_slide_list:
    description = 'window width : ' + str(window_width_hz) + ' .  window stride : ' + str(window_slide_hz)
    
    #1.Set Window width in Hz and Window Slide in Hz
    # window_width_hz = 3
    window_len = (window_width_hz/0.125)+1
    assert window_len%1 == 0
    window_len = int(window_len)
    
    # window_slide_hz = 0.5 
    window_slide_len = window_slide_hz/0.125 
    assert window_slide_len%1 == 0
    window_slide_len = int(window_slide_len)
    
    
    #2.Calculate band values and store in dataframe calculated_df
    
    # Start of the window is the middle_freq value minus window_width_hz/2 
    # End of the window is the middle freq value plus window_width_hz/2
    window_len
    
    middle_freq = []
    final_cols = []
    
    i = 0
    while i < len(frequency_vals) - window_len:
        middle_freq.append( frequency_vals[i:i+window_len].mean() )
        band_vals = single_region_df.iloc[:,i:i+window_len].mean(axis = 1) #For every row calculate the mean for the appropriate elements
        
        final_cols.append(band_vals)
        
        # scaled_psd.append( psd_values[i:i+factor].mean() )
        i+= window_slide_len
    
    calculated_df = pd.DataFrame(final_cols).T
    calculated_df.columns = middle_freq
    
    calculated_df.columns = [ str(col) + '_width_' + str(window_width_hz) for col in calculated_df.columns]
    
    
    #3.Calculate Information Gain Based on Features
    #Change to binary HC vs PD / PD+RBD ---> Generate expt specific X,y,groups 
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(calculated_df,y,groups,expt_num)

    feature_dfs.append(X_expt)
        
#5 Concatenate all feature_dfs into a single dataframe_________________________________________________________________
X_expt_concatenated = pd.concat(feature_dfs, axis=1)


# cols_list = list(X_expt_concatenated.columns)

# col_combos = []
# for col_1 in cols_list:
#     ind = cols_list.index(col_1)
#     for col_2 in cols_list[ind+1:]:
#         col_combos.append([ col_1 , col_2 ] )

# scores = []
# for col_combo in col_combos: 
#     X_selected = X_expt_concatenated[ col_combo ]
#     # clf = RidgeClassifier().fit(X_selected, y_expt)
#     clf = LinearDiscriminantAnalysis().fit(X_selected, y_expt)
#     score = accuracy_score(y_expt,  clf.predict(X_selected))
#     scores.append(score)

In [16]:
calculated_cols = []
col_names = []

X_expt_ratios = X_expt_concatenated.copy() 

for col_1 in X_expt_concatenated.columns[:20]:
    for col_2 in X_expt_concatenated.columns[:20]:
        if col_1 == col_2:
            pass
        else:
            new_col = col_1 + '/' + col_2  
            col_names.append(new_col)
            calculated_cols.append( X_expt_concatenated[col_1]/X_expt_concatenated[col_2] )
            
            X_expt_ratios[new_col] = X_expt_concatenated[col_1]/X_expt_concatenated[col_2]


len(calculated_cols)

/tmp/ipykernel_269623/187147757.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_expt_ratios[new_col] = X_expt_concatenated[col_1]/X_expt_concatenated[col_2]
/tmp/ipykernel_269623/187147757.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_expt_ratios[new_col] = X_expt_concatenated[col_1]/X_expt_concatenated[col_2]
/tmp/ipykernel_269623/187147757.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

380

In [18]:
list(X_expt_concatenated.columns)

['0.75_width_0.5',
 '1.25_width_0.5',
 '1.75_width_0.5',
 '2.25_width_0.5',
 '2.75_width_0.5',
 '3.25_width_0.5',
 '3.75_width_0.5',
 '4.25_width_0.5',
 '4.75_width_0.5',
 '5.25_width_0.5',
 '5.75_width_0.5',
 '6.25_width_0.5',
 '6.75_width_0.5',
 '7.25_width_0.5',
 '7.75_width_0.5',
 '8.25_width_0.5',
 '8.75_width_0.5',
 '9.25_width_0.5',
 '9.75_width_0.5',
 '10.25_width_0.5',
 '10.75_width_0.5',
 '11.25_width_0.5',
 '11.75_width_0.5',
 '12.25_width_0.5',
 '12.75_width_0.5',
 '13.25_width_0.5',
 '13.75_width_0.5',
 '14.25_width_0.5',
 '14.75_width_0.5',
 '15.25_width_0.5',
 '15.75_width_0.5',
 '16.25_width_0.5',
 '16.75_width_0.5',
 '17.25_width_0.5',
 '17.75_width_0.5',
 '18.25_width_0.5',
 '18.75_width_0.5',
 '19.25_width_0.5',
 '19.75_width_0.5',
 '20.25_width_0.5',
 '20.75_width_0.5',
 '21.25_width_0.5',
 '21.75_width_0.5',
 '22.25_width_0.5',
 '22.75_width_0.5',
 '23.25_width_0.5',
 '23.75_width_0.5',
 '24.25_width_0.5',
 '24.75_width_0.5',
 '25.25_width_0.5',
 '25.75_width_0.5',

In [23]:
# X_expt_ratios = X_expt_ratios.drop(columns = list(X_expt_concatenated.columns))
X_expt_ratios

,0.75_width_0.5/1.25_width_0.5,0.75_width_0.5/1.75_width_0.5,0.75_width_0.5/2.25_width_0.5,0.75_width_0.5/2.75_width_0.5,0.75_width_0.5/3.25_width_0.5,0.75_width_0.5/3.75_width_0.5,0.75_width_0.5/4.25_width_0.5,0.75_width_0.5/4.75_width_0.5,0.75_width_0.5/5.25_width_0.5,0.75_width_0.5/5.75_width_0.5,...,10.25_width_0.5/5.25_width_0.5,10.25_width_0.5/5.75_width_0.5,10.25_width_0.5/6.25_width_0.5,10.25_width_0.5/6.75_width_0.5,10.25_width_0.5/7.25_width_0.5,10.25_width_0.5/7.75_width_0.5,10.25_width_0.5/8.25_width_0.5,10.25_width_0.5/8.75_width_0.5,10.25_width_0.5/9.25_width_0.5,10.25_width_0.5/9.75_width_0.5
0,2.621641,4.271655,6.148337,9.143580,14.094370,20.998979,28.546281,34.740770,39.494906,47.784506,...,0.551454,0.667199,0.829728,1.041142,1.257434,1.410733,1.509618,1.366532,0.995608,0.884180
1,2.725338,4.111612,5.491254,8.315971,13.460734,19.522294,25.985367,31.135763,35.827674,42.428341,...,0.633866,0.750645,0.963883,1.198464,1.409104,1.560264,1.624944,1.405629,0.968258,0.799910
2,2.144560,3.093526,3.622578,5.197116,8.207705,12.729029,18.559904,24.554250,30.495338,37.580469,...,0.140212,0.172788,0.215358,0.271899,0.344578,0.438345,0.565353,0.678091,0.792922,0.858072
3,2.183823,4.213652,6.381711,8.903563,11.295113,14.592406,17.399765,20.059117,21.624531,22.131704,...,0.360058,0.368503,0.387230,0.444093,0.527107,0.637167,0.703352,0.756681,0.751501,0.797502
4,2.420587,4.645297,7.390651,10.478638,14.094284,17.611194,20.271570,23.624984,25.924114,27.311285,...,0.422240,0.444833,0.463976,0.513833,0.584093,0.700529,0.755337,0.759855,0.727678,0.756092
5,1.976530,3.131299,4.546852,7.241644,10.727115,15.127186,19.861529,25.063501,31.847579,40.156861,...,0.430337,0.542615,0.695812,0.907922,1.074305,1.223421,1.389711,1.445000,1.351485,1.185771
6,1.779079,2.613389,3.724669,5.867083,8.984923,13.000682,17.459572,21.942265,27.456392,36.659259,...,0.473964,0.632828,0.802998,1.015766,1.175527,1.433229,1.583230,1.612001,1.457550,1.129775
7,2.240775,3.352822,4.657742,6.747081,8.687091,11.126416,13.639914,16.186277,18.861481,21.650204,...,0.453410,0.520448,0.583189,0.657741,0.679411,0.650201,0.606370,0.622943,0.729081,0.838819
8,1.981493,2.902574,4.042971,5.392079,7.384299,9.408702,11.545596,13.829503,15.524283,17.090327,...,0.523966,0.576822,0.655898,0.725643,0.765978,0.741752,0.695761,0.687606,0.748216,0.851698
9,2.550850,5.117751,8.081903,11.061973,14.967871,19.779839,27.058534,34.524477,39.830354,45.711642,...,0.664943,0.763128,0.831153,0.871725,1.003908,1.121418,1.173630,1.116902,0.963715,0.882770


In [24]:
df_ratios.equals(X_expt_ratios)

False

In [31]:
X_expt_ratios['0.75_width_0.5/1.25_width_0.5']

0     2.621641
1     2.725338
2     2.144560
3     2.183823
4     2.420587
5     1.976530
6     1.779079
7     2.240775
8     1.981493
9     2.550850
10    2.507072
11    1.881247
12    2.043864
13    2.124405
14    1.743736
15    1.740105
16    1.464296
17    1.378914
18    1.935933
19    1.903405
20    2.353580
21    2.248949
22    2.416739
23    2.839407
24    1.921171
25    1.638783
50    1.663246
51    1.727523
52    1.622520
53    1.836137
54    2.013430
55    3.050807
56    2.582029
57    3.193079
58    2.773338
59    3.338518
60    3.478460
61    1.777233
62    1.421135
63    1.758414
64    1.107121
Name: 0.75_width_0.5/1.25_width_0.5, dtype: float64

In [29]:
col

('0.75_width_0.5/1.25_width_0.5',)

In [33]:
df_ratios.columns[0]

('0.75_width_0.5/1.25_width_0.5',)

In [27]:
new_colz = []
for col in df_ratios.columns:
    new_col = df_ratios[col] / X_expt_ratios[col]
    new_colz.append(new_col)

pd.concatenate(new_colz)

KeyError: ('0.75_width_0.5/1.25_width_0.5',)

In [30]:
df_ratios = pd.concat(calculated_cols, axis = 1)
df_ratios.columns = col_names
df_ratios

,0.75_width_0.5/1.25_width_0.5,0.75_width_0.5/1.75_width_0.5,0.75_width_0.5/2.25_width_0.5,0.75_width_0.5/2.75_width_0.5,0.75_width_0.5/3.25_width_0.5,0.75_width_0.5/3.75_width_0.5,0.75_width_0.5/4.25_width_0.5,0.75_width_0.5/4.75_width_0.5,0.75_width_0.5/5.25_width_0.5,0.75_width_0.5/5.75_width_0.5,...,10.25_width_0.5/5.25_width_0.5,10.25_width_0.5/5.75_width_0.5,10.25_width_0.5/6.25_width_0.5,10.25_width_0.5/6.75_width_0.5,10.25_width_0.5/7.25_width_0.5,10.25_width_0.5/7.75_width_0.5,10.25_width_0.5/8.25_width_0.5,10.25_width_0.5/8.75_width_0.5,10.25_width_0.5/9.25_width_0.5,10.25_width_0.5/9.75_width_0.5
0,2.621641,4.271655,6.148337,9.143580,14.094370,20.998979,28.546281,34.740770,39.494906,47.784506,...,0.551454,0.667199,0.829728,1.041142,1.257434,1.410733,1.509618,1.366532,0.995608,0.884180
1,2.725338,4.111612,5.491254,8.315971,13.460734,19.522294,25.985367,31.135763,35.827674,42.428341,...,0.633866,0.750645,0.963883,1.198464,1.409104,1.560264,1.624944,1.405629,0.968258,0.799910
2,2.144560,3.093526,3.622578,5.197116,8.207705,12.729029,18.559904,24.554250,30.495338,37.580469,...,0.140212,0.172788,0.215358,0.271899,0.344578,0.438345,0.565353,0.678091,0.792922,0.858072
3,2.183823,4.213652,6.381711,8.903563,11.295113,14.592406,17.399765,20.059117,21.624531,22.131704,...,0.360058,0.368503,0.387230,0.444093,0.527107,0.637167,0.703352,0.756681,0.751501,0.797502
4,2.420587,4.645297,7.390651,10.478638,14.094284,17.611194,20.271570,23.624984,25.924114,27.311285,...,0.422240,0.444833,0.463976,0.513833,0.584093,0.700529,0.755337,0.759855,0.727678,0.756092
5,1.976530,3.131299,4.546852,7.241644,10.727115,15.127186,19.861529,25.063501,31.847579,40.156861,...,0.430337,0.542615,0.695812,0.907922,1.074305,1.223421,1.389711,1.445000,1.351485,1.185771
6,1.779079,2.613389,3.724669,5.867083,8.984923,13.000682,17.459572,21.942265,27.456392,36.659259,...,0.473964,0.632828,0.802998,1.015766,1.175527,1.433229,1.583230,1.612001,1.457550,1.129775
7,2.240775,3.352822,4.657742,6.747081,8.687091,11.126416,13.639914,16.186277,18.861481,21.650204,...,0.453410,0.520448,0.583189,0.657741,0.679411,0.650201,0.606370,0.622943,0.729081,0.838819
8,1.981493,2.902574,4.042971,5.392079,7.384299,9.408702,11.545596,13.829503,15.524283,17.090327,...,0.523966,0.576822,0.655898,0.725643,0.765978,0.741752,0.695761,0.687606,0.748216,0.851698
9,2.550850,5.117751,8.081903,11.061973,14.967871,19.779839,27.058534,34.524477,39.830354,45.711642,...,0.664943,0.763128,0.831153,0.871725,1.003908,1.121418,1.173630,1.116902,0.963715,0.882770


In [5]:
reduced_slide_list = [ (0.5, 0.5), (1, 0.5), (2, 0.5), (3, 0.5), (4, 0.5), (6, 0.5), (8, 0.5), (10, 1), (12, 1), (14, 1)]
super_reduced_slide_list = [ (0.5, 0.5), (1, 0.5), (2, 0.5), (4, 1), (6, 1), (8, 1), (10,1), (14, 1) ] 

In [36]:
(388*388)-388

150156

In [17]:
# Only 127,806 ! 